# Run natural only forcing

The most important thing here is time-matching

In [ ]:
import numpy as np
import pandas as pd
import scipy.interpolate

from fair21.exceptions import ExternalForcingError

In [ ]:
def forcing_from_input(data, time, column=None, **kwargs):
    """Creates an effective_radiative_forcing input from a DataFrame or numpy array.
    
    Parameters
    ----------
    data : str, DataFrame or ndarray
        Input forcing data.
        If ndarray it must be two columns, the first being time and the second being the 
        ERF on these time steps. If DataFrame, then the time should be the index, with the ERF being either the
        sole data column, or the column to extract specified with the `column` parameter.
    time : ndarray
        Array of times to extract
    column : str or None, optional
        column to extract from `data` if a DataFrame is provided.
    
    Other parameters
    ----------------
    see `scipy.interpolate.interp1d`.
    
    Raises
    ------
    ExternalForcingError:
        if forcing cannot be determined from input data.
    
    Returns
    -------
    forcing : ndarray
        array of effective radiative forcing.
    """
    if isinstance(data, str):
        data = pd.read_csv(data, index_col=0)
    if isinstance(data, pd.DataFrame):
        base_time = data.index.values
        if column is not None:
            base_data = data.loc[:,column].values.squeeze()
        else:
            if data.shape[1] != 1:
                raise ExternalForcingError("If not providing a column name to the forcing DataFrame, then only one column other than the time index should be provided.")
            base_data = data.iloc[:,0].values.squeeze()
    elif isinstance(data, np.ndarray):
        if data.shape[1] != 2:
            raise ExternalForcingError("If specifying a numpy array of external forcing, a (n_timestep, 2) shape array is required where the first column is time and the second column is forcing. The shape of the array you provided is {}".format(data.shape))
        base_time = data[:,0]
        base_data = data[:,1]
    else:
        raise ExternalForcingError("Please provide a filepath (to a csv), a DataFrame or a numpy array")
    
    interp_func = scipy.interpolate.interp1d(base_time, base_data, **kwargs)
    data_out = interp_func(time)
    return data_out

In [ ]:
time = np.arange(1750.5, 2020)

solar_erf = pd.read_csv('../src/fair21/defaults/data/ar6/solar_erf.csv', index_col=0)
volcanic_erf = pd.read_csv('../src/fair21/defaults/data/ar6/volcanic_erf.csv', index_col=0)

In [ ]:
solar_erf.index.values

In [ ]:
solar_erf.loc[:,'solar_erf'].values.squeeze()

In [ ]:
forcing_from_input('../src/fair21/defaults/data/ar6/volcanic_erf.csv', np.arange(1750.5, 2021))

In [ ]:
forcing_from_input(
    '../src/fair21/defaults/data/ar6/volcanic_erf.csv', 
    np.arange(1750.5, 2040), 
    bounds_error=False
)

In [ ]:
forcing_from_input(
    '../src/fair21/defaults/data/ar6/volcanic_erf.csv', 
    np.arange(1750, 2040, 1/12), 
    bounds_error=False,
    fill_value=0
)

In [ ]:
forcing_from_input(
    np.array([[1750, 0], [1850, 0.3], [1950, 1.3], [2050, 4.5]]),
    np.arange(1750.5, 2100),
    bounds_error=False,
    fill_value='extrapolate'
)

In [ ]:
forcing_from_input(
    np.array([[1750, 1850, 1950, 2050], [0, 0.3, 1.3, 4.5]]),
    np.arange(1750.5, 2100),
    bounds_error=False,
    fill_value='extrapolate'
)